In [6]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import concatenate, add, Lambda
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from helpers import TrainingMonitor
from helpers import Utils

In [8]:
output_path = "../output/"

In [9]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [10]:
db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [29]:
class INCEPTION_RESNET:
    @staticmethod
    def conv_module(x, num_filter, size_filter, stride, reg, name=None):
        (convName, bnName, actName) = (None, None, None)
        if name is not None:
            convName = name + "_conv"
            bnName = name + "_bn"
            actName = name + "_act"
            
        x = Conv2D(num_filter, size_filter, strides=stride, padding="same", kernel_regularizer=l2(reg), name=convName)(x)
        x = Activation("relu", name=actName)(x)
        x = BatchNormalization(axis=-1, name=bnName)(x)
        return x

    @staticmethod
    def inception_module(x, num1x1, num3x3Reduce, num3x3, num5x5Reduce, num5x5, num1x1Proj, reg, stage):
        shortcut = x
        
        first = INCEPTION_RESNET.conv_module(x, num1x1, 1, 1, reg, name=stage + "_1x1")

        second = INCEPTION_RESNET.conv_module(x, num3x3Reduce, 1, 1, reg, name=stage + "_3x3_1")
        second = INCEPTION_RESNET.conv_module(second, num3x3, 3, 1, reg, name=stage + "_3x3_2")

        third = INCEPTION_RESNET.conv_module(x, num5x5Reduce, 1, 1, reg, name=stage + "_5x5_1")
        third = INCEPTION_RESNET.conv_module(third, num5x5, 5, 1, reg, name=stage + "_5x5_2")

        fourth = MaxPooling2D(3, strides=1, padding="same", name=stage + "_pool_1")(x)
        fourth = INCEPTION_RESNET.conv_module(fourth, num1x1Proj, 1, 1, reg, name=stage + "_pool_2")

        mixed = concatenate([first, second, third, fourth], axis=-1, name=stage + "_mixed")
        
        shortcut = Conv2D((num1x1 + num3x3 + num5x5 + num1x1Proj), 
                          1, strides=1, use_bias=False, kernel_regularizer=l2(reg))(x)
        
        x = add([mixed, shortcut])
        return x
    
    @staticmethod
    def build(include_top=True, reg=0.0005):
        inputs = Input(shape=x_train_rgbmean.shape[1:])
        
        x = INCEPTION_RESNET.inception_module(inputs, 32, 32, 32, 16, 16, 32, reg, "block1")
        x = INCEPTION_RESNET.inception_module(x, 64, 64, 64, 32, 32, 64, reg, "block2")
        x = INCEPTION_RESNET.inception_module(x, 64, 64, 64, 32, 32, 64, reg, "block3")
        
        if include_top:
            x = AveragePooling2D((8, 8))(x)
            x = Dropout(0.5, name="do")(x)
            x = Flatten(name="flatten")(x)
            x = Dense(10, kernel_regularizer=l2(reg), name="labels")(x)
            x = Activation("softmax", name="softmax")(x)
            
        model = Model(inputs, x)
        return model

In [30]:
model = INCEPTION_RESNET.build(include_top=True, reg=0.0005)

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_3x3_1_conv (Conv2D)      (None, 32, 32, 32)   128         input_4[0][0]                    
__________________________________________________________________________________________________
block1_5x5_1_conv (Conv2D)      (None, 32, 32, 16)   64          input_4[0][0]                    
__________________________________________________________________________________________________
block1_3x3_1_act (Activation)   (None, 32, 32, 32)   0           block1_3x3_1_conv[0][0]          
__________________________________________________________________________________________________
block1_5x5

In [32]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-inception-resnet.png', show_shapes=True, show_layer_names=True)

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
#filepath=output_path + "progress/inception-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
print(os.getpid())
filepath=output_path + "progress/inception-resnet-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=10)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

2604


In [40]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=20,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

Train on 33500 samples, validate on 16500 samples
Epoch 1/20
33500/33500 [==============================] - 169s 5ms/step - loss: 0.1883 - acc: 0.9443 - val_loss: 0.1928 - val_acc: 0.9427
Epoch 2/20
33500/33500 [==============================] - 185s 6ms/step - loss: 0.1844 - acc: 0.9452 - val_loss: 0.1823 - val_acc: 0.9459
Epoch 3/20
33500/33500 [==============================] - 184s 5ms/step - loss: 0.1812 - acc: 0.9459 - val_loss: 0.1880 - val_acc: 0.9434
Epoch 4/20
33500/33500 [==============================] - 185s 6ms/step - loss: 0.1788 - acc: 0.9468 - val_loss: 0.1929 - val_acc: 0.9412
Epoch 5/20
33500/33500 [==============================] - 184s 5ms/step - loss: 0.1757 - acc: 0.9479 - val_loss: 0.1920 - val_acc: 0.9421
Epoch 6/20
33500/33500 [==============================] - 185s 6ms/step - loss: 0.1729 - acc: 0.9486 - val_loss: 0.1876 - val_acc: 0.9433
Epoch 7/20
33500/33500 [==============================] - 185s 6ms/step - loss: 0.1701 - acc: 0.9504 - val_loss: 0.1847 - 

In [64]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 92.19%; Val: 92.19%; Test: 92.23%


In [41]:
model_json = model.to_json()
with open(output_path + "cifar_model_inception_resnet_96.01.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(output_path + "cifar_weight_inception_resnet_96.01.hdf5")